# WHOBS-IE-100
## Notionally Optimal, 100% Wind+Hydrogen+Other+Battery+Solar Electricity System for Ireland (IE)

- **Project:** [OESM-IE](http://ecrn.eeng.dcu.ie/projects/oesm-ie)
- **Funding:** [Sustainable Energy Authority of Ireland (SEAI) Research, Development and Demonstration Programme](https://www.seai.ie/grants/research-funding/research-development-and-demonstration-fund/), award reference SEAI RDD/00246 2018.
- **Author:** Barry McMullin, barry.mcmullin@dcu.ie
- **Last modified:** 27 May 2020
- **© 2020:** [Dublin City University](http://www.dcu.ie/)
- **Licence:** [GNU GENERAL PUBLIC LICENSE Version 3](https://www.gnu.org/licenses/gpl-3.0.en.html)

This is a derived from: [Optimal Wind+Hydrogen+Other+Battery+Solar (WHOBS) electricity systems for European countries](https://github.com/PyPSA/WHOBS)

Download the [Jupyter notebook](https://jupyter.org/) at: **TODO**


## Introduction/motivation

[WHOBS](https://github.com/PyPSA/WHOBS), and the associated interactive web application [model.energy](https://model.energy/), allows modelling of notionally optimised "firm" electricity generation for a given level of (constant/"baseload") capacity, based exclusively on *variable* renewable (VRE: here limited to wind and solar) sources, coupled with hydrogen and/or battery storage (to cover "when the wind doesn't blow and the sun doesn't shine"). 

Wind and solar resource variability is configured for states within Europe using historical data from [Renewables.ninja](https://www.renewables.ninja/) ([model.energy](https://model.energy/) extends this coverage to global geographical locations using other data sources).

**WHOBS-IE-100** adapts WHOBS to model delivery of **100%** of electricity demand for one particular European country (**Ireland**), based on [historical load data](http://www.eirgridgroup.com/how-the-grid-works/renewables/) from the [Irish Transmission System Operator (TSO) eirgrid](http://www.eirgridgroup.com/).

This result illustrates the (rough) trade-off between:

- Raw VRE *overprovision* (building more VRE capacity than can be directly dispatched at all times, but meaning that more demand can be covered directly by instantaneous VRE generations)
- Dispatch down (discarding some generation when it is in excess of instantaneous load)
- Storage (storing some generation when it is in excess of instantaneous load)
    - Short term, high efficiency storage: battery
    - Long term, low efficiency storage: hydrogen

This toy model assumes a completely isolated grid (no external interconnection).


## TODO

+ Create/release as github project: add link in metadata block at top?
+ Link to the [detailed caveats in the WHOBS README](https://github.com/PyPSA/WHOBS#warnings).
+ Add more cautions/, including:
    - *multi-annual* variability in generation and load
    - constraint as well as curtailment 
    - system stability (traditional SNSP!) 
    - onshore wind only (offshore would be less variable but generally still more expensive)
    - limited/no salt cavern storage within the IE jurisdiction (there is some in NI, but [currently targetted for NG storage!](https://www.infrastrataplc.com/projects/islandmagee-energy/)
    - unclear whether the proposed (solved) wind and solar capacity is even feasible within the jurisdiction
    - certain "low carbon", "firm" generation sources are deliberately omitted: nuclear, fossil fuel with CCS and bioenergy: it is likely that a (near) zero-$CO_2$ system could be achieved at lower cost by allowing such a wider range of options
    - etc. etc...
+ Batteries are allowed a tech-generic lifetime of 25 years (in assumptions .csv) - which seems a bit generous? Though may not change the results all that much...
+ Add some (simplistic) model of interconnection: e.g. as a `StorageUnit` with `cyclic=True`, fixed `p_nom`, efficiency somewhat reflective of real IC losses (to UK/France). This is easier than trying to pick a price for a market import/export model (though it excludes nett exports as a trade opportunity...). Skates over the NI integration connection, which arguably deserves finer grained representation (given similar wind var profile).
+ Add some representation of offshore wind as well as onshore; set `p_nom_extendable_max` for onshore, based on actual SEAI estimates. Note that offshore variability pattern is expected to be different from onshore (potential high capacity factor, absent dispatch down).
+ Add some notional representation of grid losses (matching typical SEAI/Eirgrid levels?).
+ Add some crude capability of changing average load level for target year (linear, exponential, whatever...)
+ Add more flexible options on temporal resolution - not just 1 or 3 hours, but 6, 12, 24 hours?
+ Add H2P OCGT (as well as CCGT)? Let `lopf()` optimise between OCGT and CCGT (will it all go to one?).
+ Instead of a H2 steel tank storage being binary, leave it available unconditionally, but (optionally) set an `e_extendable_max` limit on H2 salt cavern storage (possibly loosely based on actual geology available in NI).
+ Add a "middle tier" H2 storage option (between steel tank and salt cavern): [loosely based on ammonia tank storage](https://ammoniaindustry.com/ammonia-for-energy-storage-economic-and-technical-analysis/)?
+ Of course, extending into heating and transport sectors is also on the agenda!
+ Refactor the "scenario" concept:
    - Incorporate `frequency` (properly, `time_resolution` or `time_step`?).
    - Incorporate dispatch priority among VRE (solar/wind) as a scenario variable
    - Currently "hydrogen" and "hydrogen steel tank" are effectively boolean scenario variables, though not expressly incorporated in the scenario mechanism/naming. Maybe make "all" the discrete technologies boolean in this sense, with a scenario tech substring something like: `WSHTB` (Wind|Solar|H2|Tank|Battery)?
    - Refactor so that generic `network` only constructed once, and then specific details varied by scenario; should yield a little performance improvement/responsivity...
+ Possibly refactor to remove the `ct` ("country"?) use? This is inherited from WHOBS, which genuinely has configuration capability for a variety of countries; but WHOBS-IE-100 really is hard-wired already for IE?
+ Figure out/refactor exactly the way the plotting works...
+ Refactor the import of third-party data files (Renewables.ninja and Eirgrid) to automate, with local caching...
+ Create a single line diagram illustration (SLD)
+ Screencast tutorial/explanation?


In [ ]:
import pypsa

import pandas as pd

idx = pd.IndexSlice

import matplotlib.pyplot as plt

from pyomo.environ import Constraint

%matplotlib inline

In [ ]:
# Set plotsize in notebook
# https://www.mikulskibartosz.name/how-to-change-plot-size-in-jupyter-notebook/
plt.rcParams["figure.figsize"] = (12,6) 

## Major configuration settings

In [ ]:
add_hydrogen = True

#years for weather data (solar is 1985-2015 inclusive, wind is 1980-2016)
year_start = 2010
year_end = 2010

#1 is hourly, 3 is 3-hourly
frequency = 12

#change to cbc or glpk for open-source solvers
solver_name = "cbc"

## Required data

### Wind and solar resource variabilities

From [Renewables.ninja Downloads](https://www.renewables.ninja/downloads):

- Solar time series "ninja_pv_europe_v1.1_sarah.csv" from [PV v1.1 Europe (.zip)](https://www.renewables.ninja/static/downloads/ninja_europe_pv_v1.1.zip)
- Wind time series "ninja_wind_europe_v1.1_current_on-offshore.csv" from [Wind v1.1 Europe (.zip)](https://www.renewables.ninja/static/downloads/ninja_europe_wind_v1.1.zip)

### IE Load (electricity demand) variability

From [eirgrid System and Renewable Data Reports](http://www.eirgridgroup.com/how-the-grid-works/renewables/):

- [System-Data-Qtr-Hourly-2018-2019.xlsx](http://www.eirgridgroup.com/site-files/library/EirGrid/System-Data-Qtr-Hourly-2018-2019.xlsx) 
- [System-Data-Qtr-Hourly-2016-2017.xlsx](http://www.eirgridgroup.com/site-files/library/EirGrid/System-Data-Qtr-Hourly-2016-2017.xlsx)
- [System-Data-Qtr-Hourly-2014-2015.xlsx](http://www.eirgridgroup.com/site-files/library/EirGrid/System-Data-Qtr-Hourly-2014-2015.xlsx)


## Read in wind and solar variability data

**TODO:** Ideally, recode this to check for local copy, and, if not available, automatically download 
and extract the required .csv from the .zip in each case; but for the moment, just assume there is are local copies of the .csv files already available.

**TODO:** An alterative to using renewables ninja (specfically for wind) would be to extract the variability data (of actual wind generation) from historical eirgrid data. This would reflect the performance of the IE wind fleet as of whatever historical date was used: which may be a good thing or a bad thing of course (since that is almost 100% onshore for the moment, it is "biased against" offshore - arguably?).


In [ ]:
#rninja_base_url = "https://www.renewables.ninja/static/downloads/"
r_ninja_base_url = 'ninja/' # Actually already downloaded...

In [ ]:
#solar_pv_zip_file = 'ninja_europe_pv_v1.1.zip'
#solar_pv_zip_url = r_ninja_base_url + solar_pv_zip_file

solar_pv_csv_file = 'ninja_pv_europe_v1.1_sarah.csv'
solar_pv_csv_url = r_ninja_base_url + solar_pv_csv_file

#read in renewables.ninja solar time series
solar_pu_raw = pd.read_csv(solar_pv_csv_url,
                       index_col=0,parse_dates=True)
solar_pu = solar_pu_raw.resample(str(frequency)+"H").mean()

In [ ]:
#wind_zip_file = 'ninja_europe_wind_v1.1.zip'
#wind_zip_url = r_ninja_base_url + wind_zip_file

wind_csv_file = 'ninja_wind_europe_v1.1_current_on-offshore.csv'
wind_csv_url = r_ninja_base_url + wind_csv_file

#read in renewables.ninja wind time series
wind_pu_raw = pd.read_csv(wind_csv_url,
                       index_col=0,parse_dates=True)
wind_pu = wind_pu_raw.resample(str(frequency)+"H").mean()

## Read in and preprocess load and generation variability data (via Ireland TSO, [EirGrid](http://www.eirgridgroup.com/))

We start with [historical data inputs from EirGrid](http://www.eirgridgroup.com/how-the-grid-works/renewables/) which show 15-minute time series for:

- wind availability
- wind generation
- total generation
- total load

broken out by:

- IE (Republic of Ireland) only
- NI (Northern Ireland) only

As this particular notebook in only relying on this data for an example of "typical" annual variability in aggregate load we select the data for **IE (Republic of Ireland) only**.

The data will also be downsampled from 15-minute to hourly-resolution (via simple averaging of each hour).

In [ ]:
# Retrieve example eirgrid system data into a pd.dataframe

# If file already available locally, can point at that; otherwise use the web url
# (i.e. uncomment one or the other of the following two statements).

#eirgrid_base_url = "http://www.eirgridgroup.com/site-files/library/EirGrid/"
eirgrid_base_url = "eirgrid/"

load_data_filename = "System-Data-Qtr-Hourly-2014-2015.xlsx"
load_data_url = eirgrid_base_url + load_data_filename

cols = ['DateTime', 'GMT Offset', "IE Demand"]

load_data_raw = pd.read_excel(load_data_url, usecols = cols)

In [ ]:
#print(load_data_raw.dtypes)
print(load_data_raw)

## Fix the timestamps...

The raw eirgrid data has one column showing localtime (`DateTime`, type `pd.Timestamp`, holding "naive" timestamps - no recorded timezone) and a separate column showing the offset, in hours, from UTC for each individual row (`GMT Offset`). It will be simpler here to convert all the `DateTime` values to UTC (and explicitly having the UTC timezone).

We can then dispense with the `GMT Offset` column as it is redundant.

In [ ]:
from datetime import timedelta

def tz_fix(row):
  try:
    naive_timestamp = row['DateTime']
    gmt_offset = row['GMT Offset'] 
    utc_timestamp = naive_timestamp - timedelta(hours=float(gmt_offset))
        # float() conversion required for timedelta() argument!
        # Must SUBTRACT the GMT Offset to get GMT/UTC
    row['DateTime'] = utc_timestamp.tz_localize('UTC')
  except Exception as inst:
    print(F"Exception:\n {row}")
    print(inst)
  return row

# This may be rather be slow for a big dataset...
load_data_raw = load_data_raw.apply(tz_fix, axis=1).drop(columns='GMT Offset')

load_data_raw.set_index('DateTime', verify_integrity=True, inplace=True)

## Extract period of interest

In [ ]:
load_data_raw = load_data_raw.loc['2015-01-01 00:00:00+0':'2015-12-31 23:45:00+0']

print(load_data_raw)

load_raw = load_data_raw.loc[:,'IE Demand'] # convert to pd.Series

load_raw_max = load_raw.max()
load_raw_mean = load_raw.mean()
load_raw_min = load_raw.min()
load_raw_e = load_raw.sum()*0.25 # Raw time interval is 15m == 0.25h

print(F"load_raw_max: {(load_raw_max/1.0e3) : 6.3f} GW")
print(F"load_raw_mean: {(load_raw_mean/1.0e3) : 6.3f} GW")
print(F"load_raw_min: {(load_raw_min/1.0e3) : 6.3f} GW")
print(F"load_raw_e: {(load_raw_e/1.0e6) : 6.3f} TWh")

## FIXME! Data quality checks?

Before going further, should really do some data quality checks - at least make sure we don't have missing/`NaN` values in the numeric timeseries data? But for now, this omission is just logged for future reference...

## Configure the load timeseries

This is derived from the IE empirical/historical `Demand`. 

In [ ]:
load_data = load_data_raw.resample(str(frequency)+"H").mean()

load = load_data.loc[:,'IE Demand'] # convert to pd.Series

load_max = load.max()
load_mean = load.mean()
load_min = load.min()
load_e = load.sum()*frequency

In [ ]:
print(load)

print(F"load_max: {(load_max/1.0e3) : 6.3f} GW")
print(F"load_mean: {(load_mean/1.0e3) : 6.3f} GW")
print(F"load_min: {(load_min/1.0e3) : 6.3f} GW")
print(F"load_e: {(load_e/1.0e6) : 6.3f} TWh")

## Required functions

In [ ]:
# TODO: need some docs/explanation/sources for this calculation?

def annuity(lifetime, rate):
    if rate == 0.0 :
        return 1.0/lifetime
    else:
        return rate/(1.0 - (1.0 / (1.0 + rate)**lifetime))

In [ ]:
def prepare_assumptions(Nyears=1,usd_to_eur=1/1.2,assumptions_year=2020):
    """set all asset assumptions and other parameters"""

    assumptions = pd.read_csv("assumptions.csv",index_col=list(range(3))).sort_index()

    #correct units to MW and EUR
    assumptions.loc[assumptions.unit.str.contains("/kW"),"value"]*=1e3
    assumptions.loc[assumptions.unit.str.contains("USD"),"value"]*=usd_to_eur

    assumptions = assumptions.loc[idx[:,assumptions_year,:],
                                  "value"].unstack(level=2).groupby(level="technology").sum(min_count=1)

    #fill defaults
    assumptions = assumptions.fillna({"FOM" : assumptions.at["default","FOM"],
                                      "discount rate" : assumptions.at["default","discount rate"],
                                      "lifetime" : assumptions.at["default","lifetime"]})

    #annualise investment costs, add FOM
    assumptions["fixed"] = [(annuity(v["lifetime"],v["discount rate"]) + 
                             v["FOM"]/100.)*v["investment"]*Nyears for i,v in assumptions.iterrows()]

    return assumptions

In [ ]:
def solve_network(ct,scenario):

    Nyears = year_end - year_start + 1

    assumptions_year = int(scenario[:4])

    assumptions = prepare_assumptions(Nyears=Nyears,
                                      assumptions_year=assumptions_year)

    if "steel_tanks" in scenario:
        assumptions.loc["H2 storage"] = assumptions.loc["H2 steel tank storage"]
    else:
        assumptions.loc["H2 storage"] = assumptions.loc["H2 underground storage"]


    network = pypsa.Network()

    snapshots = pd.date_range("{}-01-01".format(year_start),"{}-12-31 23:00".format(year_end),
                              freq=str(frequency)+"H")

    network.set_snapshots(snapshots)

    network.snapshot_weightings = pd.Series(float(frequency),index=network.snapshots)

    network.add("Bus",ct)
    network.add("Load",ct,
                bus=ct,
                p_set= load.values)

    network.add("Generator",ct+" solar",
                bus=ct,
                p_max_pu = solar_pu[ct],
                p_nom_extendable = True,
                marginal_cost = 0.02, 
                #Small cost to prefer curtailment to destroying energy in storage, wind curtails before solar
                capital_cost = assumptions.at['utility solar PV','fixed'],
                #p_nom_max = 0.0
               )

    network.add("Generator",ct+" wind",
                bus=ct,
                p_max_pu = wind_pu[ct+"_ON"],
                p_nom_extendable = True,
                marginal_cost = 0.01, 
                #Small cost to prefer curtailment to destroying energy in storage, wind curtails before solar
                capital_cost = assumptions.at['onshore wind','fixed'])

    network.add("Bus",ct + " battery")

    network.add("Store",ct + " battery storage",
                bus = ct + " battery",
                e_nom_extendable = True,
                e_cyclic=True,
                capital_cost=assumptions.at['battery storage','fixed'])

    network.add("Link",ct + " battery charge",
                bus0 = ct,
                bus1 = ct + " battery",
                efficiency = assumptions.at['battery inverter','efficiency'],
                p_nom_extendable = True,
                capital_cost=assumptions.at['battery inverter','fixed'])

    network.add("Link",ct + " battery discharge",
                bus0 = ct + " battery",
                bus1 = ct,
                p_nom_extendable = True,
                efficiency = assumptions.at['battery inverter','efficiency'])

    def extra_functionality(network,snapshots):
        def battery(model):
            return (model.link_p_nom[ct + " battery charge"] 
                    == model.link_p_nom[ct + " battery discharge"]*network.links.at[ct 
                    + " battery charge","efficiency"])

        network.model.battery = Constraint(rule=battery)

    if add_hydrogen:

        network.add("Bus",
                     ct + " H2",
                     carrier="H2")

        network.add("Link",
                    ct + " H2 electrolysis",
                    bus1=ct + " H2",
                    bus0=ct,
                    p_nom_extendable=True,
                    efficiency=assumptions.at["H2 electrolysis","efficiency"],
                    capital_cost=assumptions.at["H2 electrolysis","fixed"])

        network.add("Link",
                     ct + " H2 to power",
                     bus0=ct + " H2",
                     bus1=ct,
                     p_nom_extendable=True,
                     efficiency=assumptions.at["H2 CCGT","efficiency"],
                     capital_cost=assumptions.at["H2 CCGT","fixed"]*assumptions.at["H2 CCGT","efficiency"])  
                     #NB: fixed cost is per MWel

        network.add("Store",
                     ct + " H2 storage",
                     bus=ct + " H2",
                     e_nom_extendable=True,
                     e_cyclic=True,
                     capital_cost=assumptions.at["H2 storage","fixed"])


    if solver_name == "gurobi":
        solver_options = {"threads" : 4,
                          "method" : 2,
                          "crossover" : 0,
                          "BarConvTol": 1.e-5,
                          "FeasibilityTol": 1.e-6 }
    else:
        solver_options = {}


    network.consistency_check()

    network.lopf(solver_name=solver_name,
                 solver_options=solver_options,
                 extra_functionality=extra_functionality)

    return network

In [ ]:
def gather_stats(stats,network,ct,full_scenario):
    
    stats.loc["cost (€/MWh)",full_scenario] = network.buses_t.marginal_price.mean()[ct]
    
    gens = ["wind","solar"]
    
    for g in gens:
        stats.loc[g+" generation (MW)",full_scenario] = network.generators.p_nom_opt[ct + " " + g]
    stats.loc["Battery charge/discharge (MW)",full_scenario] = network.links.p_nom_opt[ct + " battery charge"]
        # NB: battery charge/discharge p_nom constrainted to match, modulo efficiency
    stats.loc["Battery storage (MWh)",full_scenario] = network.stores.e_nom_opt[ct + " battery storage"]
    stats.loc["P2H (electolysis, MW)",full_scenario] = network.links.p_nom_opt[ct + " H2 electrolysis"]
    stats.loc["H2P (H2 CCGT, MW)",full_scenario] = network.links.p_nom_opt[ct + " H2 to power"]
    stats.loc["H2 storage (MWh)",full_scenario] = network.stores.e_nom_opt[ct + " H2 storage"]
    available = network.generators_t.p_max_pu.multiply(network.generators.p_nom_opt).sum() * frequency
    used = network.generators_t.p.sum() * frequency
    curtailment = (available-used)/available
    
    total_hours = network.snapshot_weightings.sum()
    
    stats.loc["Wind dispatch down (%)",full_scenario] = curtailment[ct + " wind"]
    stats.loc["Wind capacity factor (%)",full_scenario] = (
        used[ct + " wind"]/(network.generators.p_nom_opt[ct + " wind"]*total_hours))
    
    stats.loc["Solar dispatch down (%)",full_scenario] = curtailment[ct + " solar"]
    stats.loc["Solar capacity factor (%)",full_scenario] = (
        used[ct + " solar"]/(network.generators.p_nom_opt[ct + " solar"]*total_hours))
    
    return stats

## Solve the system

In [ ]:
ct = "IE"
#scenario = "2030-steel_tanks"
scenario = "2030"
full_scenario = ct+'-'+scenario+'-WP2010'+'-'+str(frequency)+"H"

network = solve_network(ct, scenario)

In [ ]:
hours = network.snapshot_weightings.sum()
print(F"Total hours: {hours : 6.0f}")
assert(hours == network.snapshot_weightings.count()*frequency)

In [ ]:
p_nom_wind = network.generators.p_nom_opt[ct + " wind"]
print(F"p_nom_wind: {(p_nom_wind/1e3) : 6.3f} GW") 

e_nom_wind = p_nom_wind * hours
print(F"e_nom_wind : {(e_nom_wind/1e6) : 6.3f} TWh")

cap_fact_wind_max = network.generators_t.p_max_pu[ct + " wind"].mean() 
print(F"cap_fact_max: {(cap_fact_wind_max *100.0) : 6.1f}%") 

#available = network.generators_t.p_max_pu.multiply(network.generators.p_nom_opt).sum()
e_available_wind = cap_fact_wind_max * e_nom_wind
print(F"e_available_wind : {(e_available_wind/1e6) : 6.3f} TWh")

e_used_wind = network.generators_t.p[ct + " wind"].sum()*frequency
print(F"e_used_wind : {(e_used_wind/1e6) : 6.3f} TWh")

cap_fact_wind = e_used_wind/e_nom_wind
print(F"cap_fact_wind: {(cap_fact_wind *100.0) : 6.1f}%")

#stats.loc["Wind capacity factor (%)",full_scenario] = (
#        used[ct + " wind"]/(network.generators.p_nom_opt[ct + " wind"]*network.snapshot_weightings.sum()))

dispatch_down_wind = (e_available_wind - e_used_wind)/e_available_wind
print(F"dispatch_down_wind: {(dispatch_down_wind *100.0) : 6.3f}%")

In [ ]:
p_nom_solar = network.generators.p_nom_opt[ct + " solar"]
print(F"p_nom_solar: {(p_nom_solar/1e3) : 6.3f} GW") 

e_nom_solar = p_nom_solar * hours
print(F"e_nom_solar : {(e_nom_solar/1e6) : 6.3f} TWh")

cap_fact_solar_max = network.generators_t.p_max_pu[ct + " solar"].mean() 
print(F"cap_fact_max: {(cap_fact_solar_max *100.0) : 6.1f}%") 

#available = network.generators_t.p_max_pu.multiply(network.generators.p_nom_opt).sum()
e_available_solar = cap_fact_solar_max * e_nom_solar
print(F"e_available_solar : {(e_available_solar/1e6) : 6.3f} TWh")

e_used_solar = network.generators_t.p[ct + " solar"].sum()*frequency
print(F"e_used_solar : {(e_used_solar/1e6) : 6.3f} TWh")

cap_fact_solar = e_used_solar/e_nom_solar
print(F"cap_fact_solar: {(cap_fact_solar *100.0) : 6.1f}%")

#stats.loc["solar capacity factor (%)",full_scenario] = (
#        used[ct + " solar"]/(network.generators.p_nom_opt[ct + " solar"]*network.snapshot_weightings.sum()))

dispatch_down_solar = (e_available_solar - e_used_solar)/e_available_solar
print(F"dispatch_down_solar: {(dispatch_down_solar *100.0) : 6.3f}%")

In [ ]:
e_used = e_used_solar + e_used_wind
sys_storage_efficiency = load_e/e_used 
  # Losses arise from storage round-trip/temporal shifting
  # Excludes "dispatch down" - though that should arguably be counted also?
print(F"sys_storage_efficiency: {(sys_storage_efficiency *100.0) : 6.1f}%")

e_available = e_available_wind + e_available_solar
sys_overall_efficiency = load_e/e_available
print(F"sys_overall_efficiency: {(sys_overall_efficiency *100.0) : 6.1f}%")

penetration_wind = e_used_wind/e_used
print(F"penetration_wind: {(penetration_wind *100.0) : 6.1f}%")

penetration_solar = e_used_solar/e_used
print(F"penetration_solar: {(penetration_solar *100.0) : 6.1f}%")


In [ ]:
# Uncomment and run this if/when desired to clear the accumulated stats.
# FIXME: would like to have any way to shovel these result checkpoints out to 
# a external (persistent) data files?

# stats_checkpoint_0_pre_VRE_freq_mean = stats # manually fix/choose archive name!
# stats=pd.DataFrame() 

In [ ]:
try: stats
except NameError: 
    stats=pd.DataFrame()

gather_stats(stats,network,ct,full_scenario)

In [ ]:
def fmt_percent(x) :
    return (F"{x : 6.2%}")

def fmt_GW(x) : 
    # x in MW
    return (F"{x/1.0e3 : 6.2f} GW")

def fmt_TWh(x) : 
    # x in TWh
    return (F"{x/1.0e6 : 6.2f} TWh")

fmt_funcs = {
    'wind generation (MW)' : fmt_GW,
    'H2 storage (MWh)' : fmt_TWh,
    'Wind dispatch down (%)': fmt_percent,
    }

In [ ]:
fmt_stats=stats.transform(func=fmt_funcs, axis='columns')
display(fmt_stats)

In [ ]:
#stats.style.format("{:10.2f}")
#print(stats)
#stats.apply(lambda x: type(x))

#pd.set_option('precision', 10)
#pd.set_eng_float_format(accuracy=3, use_eng_prefix=True)

#def tst_format(x) :
#    return (F"{x : 10.2f}")

#stats_str=stats.apply(lambda x: x.to_string(float_format=tst_format))

#def tst_format(row) :
    #print(type(x))
    #print(x.name)
#    if (row.name == 'Wind dispatch down (%)') :
#        row = row.apply(lambda x : F"{x*100 : 10.2f}")
#        display(row)
#    return row

#stats_str=stats.apply(func = tst_format, axis='columns')
#stats_str=stats.transform(func={'Wind dispatch down (%)': tst_format}, axis='columns')
#stats_str=stats.transform(func={'Wind dispatch down (%)': (lambda x : F"{x*100 : 10.2f}%")}, axis='columns')

#stats_str
#print(type(stats_str))

#print(stats_str)

#tst = stats.loc['cost (€/MWh)',:]
#print(tst)
#print(tst.name)
#print(tst.dtype)
#print(tst.axes)
#print(tst.index)
#print(tst.values)
#print(tst.to_latex())
#print(tst.to_string(float_format=tst_format))

In [ ]:
stats_checkpoint_0_pre_VRE_freq_mean 
 # Archived before fixing the VRE pu data to re-sample (taking means) rather than 
 # just sub-sample (per original WHOBS)

In [ ]:
#absolute market value in EUR/MWh
(network.generators_t.p.multiply(network.buses_t.marginal_price[ct],axis=0).sum()/network.generators_t.p.sum())

In [ ]:
#market cost in EUR/MWh (LCOE) - for long-term equilibrium without additional constraints, same as market value
(network.generators.capital_cost*network.generators.p_nom_opt)/network.generators_t.p.sum()

In [ ]:
#relative market value
(network.generators_t.p.multiply(network.buses_t.marginal_price[ct],axis=0).sum()/
     network.generators_t.p.sum())/network.buses_t.marginal_price[ct].mean()

In [ ]:
#relative market value
(network.links_t.p0.multiply(network.buses_t.marginal_price[ct],axis=0).sum()/
     network.links_t.p0.sum())/network.buses_t.marginal_price[ct].mean()

In [ ]:
%matplotlib notebook

#start = "2015-01-01"
#stop = "2015-12-31"
start = F"{year_start}-01-01"
stop = F"{year_end}-12-31"

fig, ax = plt.subplots()
#fig.set_size_inches((10,8))


rename = {"wind" : "onshore wind",
          "solar" : "utility solar PV",
          "battery discharge" : "battery discharge",
          "battery charge" : "battery charge",
          "H2 electrolysis" : "hydrogen electrolysis",
          "H2 to power" : "hydrogen turbine"}

rename = {ct + " "+k : v for k,v in rename.items()}

rename[ct] = "load"

colors = {"onshore wind" : "b",
           "utility solar PV" : "y",
           "battery discharge" : "gray",
           "battery charge" : "gray",
           "load" : "k",
           "hydrogen electrolysis" : "m",
           "hydrogen turbine" : "r"
          }


# FIXME: for some reason (solver tolerance?) some solution values that should be strictly 
# positive or negative may be infinitesimally (< 10e-10) of the other sign. This will cause 
# df.plot(stacked=True) to throw an error (requires all values to have same sign, positive or 
# negative). So we do an ugly kludge to fix it...
def zero_tol(x) :
        if ((x>-1.0e-10) and (x<1.0e-10)):
            x = 0.0
        return(x)

positive = pd.concat((network.generators_t.p,-network.links_t.p1[[ct+" H2 to power",ct+" battery discharge"]]),
                     axis=1).rename(columns=rename)
positive = positive.applymap(zero_tol)

negative = pd.concat((-network.loads_t.p,-network.links_t.p0[[ct+" H2 electrolysis",ct+" battery charge"]]),
                     axis=1).rename(columns=rename)
negative = negative.applymap(zero_tol)

print((abs(positive.sum(axis=1) + negative.sum(axis=1)) > 0.1).any())

positive.loc[start:stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                              color=[colors[i] for i in positive.columns])

negative.loc[start:stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                              color=[colors[i] for i in negative.columns])

ax.set_ylim([-2.0*load_max,+2.0*load_max])

ax.set_xlim([start,stop])

ax.set_ylabel("Dispatch (generation is +ve, demand is -ve) [MW]")

ax.legend(ncol=3,loc="upper left")

fig.tight_layout()

#fig.savefig("img/{}-{}-{}-{}.png".format(ct,scenario,start,stop),dpi=100)

In [ ]:
network.buses_t.marginal_price.plot()

In [ ]:
network.buses_t.marginal_price.mean()

In [ ]:
network.links_t.p0.mean()/network.links.p_nom_opt

In [ ]:
%matplotlib notebook

opt_costs = pd.Series()


opt_costs = pd.concat((opt_costs,network.generators.capital_cost*network.generators.p_nom_opt))

opt_costs = pd.concat((opt_costs,network.links.capital_cost*network.links.p_nom_opt))

opt_costs = pd.concat((opt_costs,network.stores.capital_cost*network.stores.e_nom_opt))


(opt_costs/opt_costs.sum()).plot(kind="bar",grid=True)


In [ ]:
print(opt_costs)

## Reflection

+ I don't really have a good intuition for why battery storage is being used at the level it is. Presumably if we go to more coarse-grained temporal resolution it stops being used altogether?
+ Battery and DSM (including heating and vehicle charging flexibilities) presumablly all fall into the same ~24 hour flexibility regime.
+ Flow batteries would, presumably, have a quite different profile: maybe conceivably competitive with ammonia? Could we add a model of that?